In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
import osmnx as ox
import pandas as pd
import numpy as np
from Data import Data 
import networkx as nx
from networkx import algorithms 
import math

/Users/ridhaalkhabaz/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/Users/ridhaalkhabaz/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#d

In [3]:
graph = ox.graph_from_address('Athens, Municipality of Athens, Regional Unit of Central Athens, Attica, 10667, Greece', network_type='drive') 

In [4]:
df = pd.read_pickle("Data_sample.pkl")
X = df.copy()

In [5]:
#get the nearest edge and node for each location:
nodes = np.zeros(len(df))
EdgeStart = np.zeros(len(df))
EdgeEnd = np.zeros(len(df))
for i in range(len(df)):
    tup1 = (df.iloc[i]['lat'], df.iloc[i]['lon'])
    nodes[i] = ox.get_nearest_node(graph, tup1 , method='euclidean')
    start, end, c = ox.get_nearest_edge(graph, tup1)
    EdgeStart[i] = start
    EdgeEnd[i] = end

In [6]:
df['Nearest Node'] = nodes
df['Nearest Edge Start'] =  EdgeStart
df['Nearest Edge End'] =  EdgeEnd

In [7]:
df

lat        lon    speed  lon_acc  lat_acc  type  traveled_d  \
id time                                                                       
1  0.00   37.977482  23.735405  34.0206  -0.0167   0.0074  Taxi      320.26   
   0.04   37.977485  23.735402  34.0197   0.0040   0.0086  Taxi      320.26   
   0.08   37.977488  23.735400  34.0211   0.0167   0.0097  Taxi      320.26   
   0.12   37.977491  23.735398  34.0245   0.0294   0.0108  Taxi      320.26   
   0.16   37.977494  23.735395  34.0305   0.0540   0.0120  Taxi      320.26   
...             ...        ...      ...      ...      ...   ...         ...   
5  29.24  37.979836  23.733313  36.8072  -3.7893   1.1652   Car      272.25   
   29.28  37.979838  23.733311  36.4768  -3.8277   1.1690   Car      272.25   
   29.32  37.979841  23.733309  36.1375  -3.8118   1.1665   Car      272.25   
   29.36  37.979844  23.733306  35.7859  -3.8818   1.1568   Car      272.25   
   29.40  37.979847  23.733304  35.4092  -4.0068   1.1387   Car      272.25   

          avg_speed  Nearest Node  Nearest Edge Start  Nearest Edge End  
id time                                                                  
1  0.00   35.366264  3.339822e+09        3.339822e+09       250691723.0  
   0.04   35.366264  3.339822e+09        3.339822e+09       250691723.0  
   0.08   35.366264  3.339822e+09        3.339822e+09       250691723.0  
   0.12   35.366264  3.339822e+09        3.339822e+09       250691723.0  
   0.16   35.366264  3.339822e+09        3.339822e+09       250691723.0  
...             ...           ...                 ...               ...  
5  29.24  33.336716  2.506989e+08        2.506989e+08       250698925.0  
   29.28  33.336716  2.506989e+08        2.506989e+08       250698925.0  
   29.32  33.336716  2.506989e+08        2.506989e+08       250698925.0  
   29.36  33.336716  2.506989e+08        2.506989e+08       250698925.0  
   29.40  33.336716  2.506989e+08        2.506989e+08       250698925.0  

[3460 rows x 11 columns]

In [8]:
def Distance_achieved(edge_node1,edge_node2,lonV, latV):
    #assuming that edge_node1 is the starting point
    lon0 = graph.nodes[edge_node1]['x']
    lon1 = graph.nodes[edge_node2]['x']
    lat0 = graph.nodes[edge_node1]['y']
    lat1 = graph.nodes[edge_node2]['y']
    a = Distance_Euc(lon0, lat0, lon1, lat1)
    b = Distance_Euc(lon0, lat0, lonV, latV)
    return b/a

In [9]:
def Distance_Euc(lon0, lat0, lon1, lat1):
    lon0 = math.radians(lon0)
    lat0 = math.radians(lat0)
    lon1 = math.radians(lon1)
    lat1 = math.radians(lat1)
    dlon = lon1 - lon0
    dlat = lat1 - lat0
    a = math.sin(dlat / 2)**2 + math.cos(lat0) * math.cos(lat1) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return 6373.0*c

In [10]:
Dis = np.zeros(len(df))
for i in range(len(df)):
    b = df.iloc[i]['lon']
    c = df.iloc[i]['lat']
    a = Distance_achieved(df.iloc[i]['Nearest Edge Start'], df.iloc[i]['Nearest Edge End'], b, c)
    Dis[i] = a

In [11]:
df['Distance Achieved'] = Dis

In [12]:
df

lat        lon    speed  lon_acc  lat_acc  type  traveled_d  \
id time                                                                       
1  0.00   37.977482  23.735405  34.0206  -0.0167   0.0074  Taxi      320.26   
   0.04   37.977485  23.735402  34.0197   0.0040   0.0086  Taxi      320.26   
   0.08   37.977488  23.735400  34.0211   0.0167   0.0097  Taxi      320.26   
   0.12   37.977491  23.735398  34.0245   0.0294   0.0108  Taxi      320.26   
   0.16   37.977494  23.735395  34.0305   0.0540   0.0120  Taxi      320.26   
...             ...        ...      ...      ...      ...   ...         ...   
5  29.24  37.979836  23.733313  36.8072  -3.7893   1.1652   Car      272.25   
   29.28  37.979838  23.733311  36.4768  -3.8277   1.1690   Car      272.25   
   29.32  37.979841  23.733309  36.1375  -3.8118   1.1665   Car      272.25   
   29.36  37.979844  23.733306  35.7859  -3.8818   1.1568   Car      272.25   
   29.40  37.979847  23.733304  35.4092  -4.0068   1.1387   Car      272.25   

          avg_speed  Nearest Node  Nearest Edge Start  Nearest Edge End  \
id time                                                                   
1  0.00   35.366264  3.339822e+09        3.339822e+09       250691723.0   
   0.04   35.366264  3.339822e+09        3.339822e+09       250691723.0   
   0.08   35.366264  3.339822e+09        3.339822e+09       250691723.0   
   0.12   35.366264  3.339822e+09        3.339822e+09       250691723.0   
   0.16   35.366264  3.339822e+09        3.339822e+09       250691723.0   
...             ...           ...                 ...               ...   
5  29.24  33.336716  2.506989e+08        2.506989e+08       250698925.0   
   29.28  33.336716  2.506989e+08        2.506989e+08       250698925.0   
   29.32  33.336716  2.506989e+08        2.506989e+08       250698925.0   
   29.36  33.336716  2.506989e+08        2.506989e+08       250698925.0   
   29.40  33.336716  2.506989e+08        2.506989e+08       250698925.0   

          Distance Achieved  
id time                      
1  0.00            0.490988  
   0.04            0.493622  
   0.08            0.495962  
   0.12            0.498301  
   0.16            0.500935  
...                     ...  
5  29.24           0.508215  
   29.28           0.510234  
   29.32           0.512874  
   29.36           0.515903  
   29.40           0.518543  

[3460 rows x 12 columns]